# ４次元配列

#### 高さ28・横幅28で1チャンネルのデータが10個ある場合

In [4]:
import numpy as np

x = np.random.rand(10, 1, 28, 28) # ランダムにデータを生成
x.shape

(10, 1, 28, 28)

#### 各データへのアクセス

In [6]:
x[0].shape # (1, 28, 28)
x[1].shape # (1, 28, 28) 

(1, 28, 28)

#### データの1チャンネル目の空間データにアクセス（28×28の数値データ）

In [7]:
x[0, 0] # もしくは x[0][0]

array([[0.97020214, 0.91463794, 0.43741578, 0.15137601, 0.28266198,
        0.79503946, 0.60324284, 0.35155465, 0.83660588, 0.35181556,
        0.44480022, 0.53954861, 0.99450197, 0.18998135, 0.381965  ,
        0.22610977, 0.8328797 , 0.15387826, 0.81959077, 0.73081432,
        0.82835099, 0.06053113, 0.22591576, 0.811113  , 0.42295761,
        0.35073203, 0.88570232, 0.28550079],
       [0.36862906, 0.42621914, 0.54707829, 0.87292   , 0.55280619,
        0.26964344, 0.61508364, 0.9147639 , 0.14081584, 0.01225429,
        0.8753357 , 0.97815308, 0.21906155, 0.2068474 , 0.21159562,
        0.24419141, 0.37600386, 0.6768068 , 0.64740537, 0.79461506,
        0.07423679, 0.1664924 , 0.586301  , 0.97812874, 0.71345373,
        0.22550259, 0.43047251, 0.73351226],
       [0.93184966, 0.39514571, 0.48941687, 0.29380096, 0.12512171,
        0.25203819, 0.66227238, 0.34089919, 0.98810685, 0.82135633,
        0.9450434 , 0.89598144, 0.34320045, 0.82652542, 0.08998041,
        0.46527965, 0.3974

# im2col

#### データ数 N=1

In [12]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

(9, 75)


#### N=10

In [6]:
x2 = np.random.rand(10, 3, 7, 7) # 10 個のデータ
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(90, 75)


# 畳み込み層の実装

In [11]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # フィルターの展開
        out = np.dot(col, col_W) + self.b

        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        return out

# プーリング層の実装

In [13]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
       
        # 展開 (1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        # 最大値 (2)
        out = np.max(col, axis=1)
        # 整形 (3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out

# CNNの実装

In [ ]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient


class SimpleConvNet:
    """単純なConvNet

    conv - relu - pool - affine - relu - affine - softmax
    
    Parameters
    ----------
    input_size : 入力サイズ（MNISTの場合は784）
    hidden_size_list : 隠れ層のニューロンの数のリスト（e.g. [100, 100, 100]）
    output_size : 出力サイズ（MNISTの場合は10）
    activation : 'relu' or 'sigmoid'
    weight_init_std : 重みの標準偏差を指定（e.g. 0.01）
        'relu'または'he'を指定した場合は「Heの初期値」を設定
        'sigmoid'または'xavier'を指定した場合は「Xavierの初期値」を設定
    """
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        """損失関数を求める
        引数のxは入力データ、tは教師ラベル
        """
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        """勾配を求める（数値微分）

        Parameters
        ----------
        x : 入力データ
        t : 教師ラベル

        Returns
        -------
        各層の勾配を持ったディクショナリ変数
            grads['W1']、grads['W2']、...は各層の重み
            grads['b1']、grads['b2']、...は各層のバイアス
        """
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        """勾配を求める（誤差逆伝搬法）

        Parameters
        ----------
        x : 入力データ
        t : 教師ラベル

        Returns
        -------
        各層の勾配を持ったディクショナリ変数
            grads['W1']、grads['W2']、...は各層の重み
            grads['b1']、grads['b2']、...は各層のバイアス
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
        
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + s

# MNISTデータセットの学習

In [ ]:
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from ch07.simple_convnet import SimpleConvNet
from common.trainer import Trainer

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 処理に時間のかかる場合はデータを削減 
#x_train, t_train = x_train[:5000], t_train[:5000]
#x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# パラメータの保存
network.save_params("params.pkl")
print("Saved Network Parameters!")

# グラフの描画
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

train loss:2.2994001891194764
=== epoch:1, train acc:0.13, test acc:0.159 ===
train loss:2.297814015954517
train loss:2.2950924509586317
train loss:2.2885327193721046
train loss:2.2799573250780822
train loss:2.2640110831953155
train loss:2.261228398848327
train loss:2.2365422848433387
train loss:2.2294981716165587
train loss:2.1950187134375385
train loss:2.1965935627639226
train loss:2.1099178414890876
train loss:2.1064982888403323
train loss:2.054092943180061
train loss:1.9632511668897152
train loss:1.8963152604247475
train loss:1.9150136862532312
train loss:1.7894725528971096
train loss:1.8013235451000296
train loss:1.685221812420436
train loss:1.5789070219094612
train loss:1.5556490365780857
train loss:1.4110977589898988
train loss:1.3238883870400004
train loss:1.2787565838375603
train loss:1.117632479662609
train loss:1.149418127709431
train loss:0.9561359581505579
train loss:1.1105386906726316
train loss:1.0872682888583436
train loss:0.9150096133998938
train loss:0.931565358414149

train loss:0.1742728776809988
train loss:0.12610204254212173
train loss:0.15895788186310555
train loss:0.12999552681884416
train loss:0.29616475310021867
train loss:0.23781786537014502
train loss:0.15215144935214367
train loss:0.1999396902187986
train loss:0.21868169650402325
train loss:0.16961061638817868
train loss:0.3322010839303076
train loss:0.14170837780998496
train loss:0.1567882138050212
train loss:0.20587788958188571
train loss:0.2787076679370055
train loss:0.3091801593089792
train loss:0.30391584071397704
train loss:0.09611251656736192
train loss:0.25916063784076604
train loss:0.27155509374110226
train loss:0.23004212291150597
train loss:0.3230843583274114
train loss:0.13451558485154
train loss:0.13074046340618437
train loss:0.1935799132597757
train loss:0.21270017305198954
train loss:0.2176885598647622
train loss:0.13008134744984073
train loss:0.23974257536515967
train loss:0.2583583624371088
train loss:0.2627641201738847
train loss:0.36721097083581905
train loss:0.264648733

train loss:0.18327274436193386
train loss:0.2008103823714942
train loss:0.06466493489224781
train loss:0.11417520028340655
train loss:0.11105587336318815
train loss:0.09630032992598286
train loss:0.13488842079980587
train loss:0.14401782185619347
train loss:0.07911488986668357
train loss:0.1121754829827635
train loss:0.19281051479230038
train loss:0.06999416410450564
train loss:0.1540666239990426
train loss:0.14563985302722696
train loss:0.026330984000739487
train loss:0.15345883411059635
train loss:0.15810617404647148
train loss:0.13047374499906875
train loss:0.15610379754766973
train loss:0.15613670748825703
train loss:0.10559231973211186
train loss:0.14163655885622284
train loss:0.1089986202266242
train loss:0.17185515620445277
train loss:0.073571455927178
train loss:0.08703362999830098
train loss:0.10900117596932937
train loss:0.09664317781414629
train loss:0.11812193043053558
train loss:0.15709523609748413
train loss:0.0444238642583138
train loss:0.0641929485931773
train loss:0.09

train loss:0.08901577977202373
train loss:0.08294666020297102
train loss:0.03253126702635732
train loss:0.08708758268933295
train loss:0.0662230703657055
train loss:0.12192867993403263
train loss:0.09043517853887742
train loss:0.0750760417789266
train loss:0.036139357853967026
train loss:0.07022793395975956
train loss:0.03279719383014986
train loss:0.0900025461956139
train loss:0.15375109521992292
train loss:0.04533364454672796
train loss:0.17322236832397825
train loss:0.058263702405279366
train loss:0.046135534879216955
train loss:0.09536500110354484
train loss:0.037346034959018705
train loss:0.050366878519606466
train loss:0.08100883321755495
train loss:0.07743810110191257
train loss:0.049181977666630274
train loss:0.07123734316991819
train loss:0.07359406546038534
train loss:0.029681128594529812
train loss:0.0801249209792265
train loss:0.044947180149041414
train loss:0.06006915272734854
train loss:0.06996867567436807
train loss:0.09750901139188167
train loss:0.05010738606051751
trai

train loss:0.04298645986671582
train loss:0.03652411526304304
train loss:0.10688556087894369
train loss:0.054305691520720345
train loss:0.031984698536782104
train loss:0.0715780618484914
train loss:0.016474044986891045
train loss:0.04519445473402678
train loss:0.0267244433939947
train loss:0.030145849839506786
train loss:0.05703928058451977
train loss:0.05454472122617242
train loss:0.06531362625236818
train loss:0.06341502946251745
train loss:0.10899210843656527
train loss:0.06976538982368297
train loss:0.03930688074852568
train loss:0.02816125559836984
train loss:0.032193992792121554
train loss:0.09955261113144692
train loss:0.19307179357123172
train loss:0.038148190127012524
train loss:0.06738169393166743
train loss:0.0407869014978051
train loss:0.10934249842171724
train loss:0.015543115403701473
train loss:0.07400614913443217
train loss:0.051260420576393254
train loss:0.057938783957342775
train loss:0.07771189672799339
train loss:0.05869394843070521
train loss:0.045513844598061404
t

...

train loss:0.028882208680972386
train loss:0.018653847629672576
train loss:0.062427428723633396
train loss:0.018812917831844723
train loss:0.06203491253967673
train loss:0.023922086328333623
train loss:0.01296394725164264
train loss:0.02687361036190908
train loss:0.02228920810048889
train loss:0.009451440684093343
train loss:0.07307853195356047
train loss:0.026264416699409713
train loss:0.07948602868563
train loss:0.06138738199991537
train loss:0.06890145520327744
train loss:0.02910605890883762
train loss:0.03442119472809351
train loss:0.025994748031343794
train loss:0.006530257844337074
train loss:0.015427783718107277
train loss:0.02220630745606302
train loss:0.06050998081969474
train loss:0.06251821136395795
train loss:0.039939627417763475
train loss:0.027324187336058944
train loss:0.028205063158291854
train loss:0.06424370065518564
train loss:0.022297609043859534
train loss:0.0326877434225851
train loss:0.047609297006596635
train loss:0.08458383922701841
train loss:0.050715409280262

train loss:0.015932304398281246
train loss:0.019043211766018524
train loss:0.033040717002172135
train loss:0.04791203326679498
train loss:0.05199916088966261
train loss:0.021106633337455518
train loss:0.03409499623788679
train loss:0.005043396428806549
train loss:0.04978491144623817
train loss:0.041454420039031284
train loss:0.054252932968452516
train loss:0.016476827302783673
train loss:0.02807258449585015
train loss:0.047395221474518204
train loss:0.03334613862799065
train loss:0.026841645045167558
train loss:0.12569351746030435
train loss:0.03240914659689337
train loss:0.031135018197252187
train loss:0.008780967821110226
train loss:0.03428356633073849
train loss:0.022405206343839898
train loss:0.014075760018274297
train loss:0.1102374733717601
train loss:0.018681436829939047
train loss:0.07498510199576044
train loss:0.04919718547968218
train loss:0.06219571103662065
train loss:0.13080490367696776
train loss:0.05222160256827417
train loss:0.04854911270796229
train loss:0.034598189854

train loss:0.016104162213446807
train loss:0.02157251749903376
train loss:0.02216594129671078
train loss:0.030483322390849842
train loss:0.011139014657218957
train loss:0.03210577502061355
train loss:0.005289679880259754
train loss:0.017094507785049874
train loss:0.04591672540430396
train loss:0.015355464216352303
train loss:0.037640076136892256
train loss:0.03591835527988272
train loss:0.03283953071675771
train loss:0.13152249238273597
train loss:0.008485231071725441
train loss:0.02965712605882105
train loss:0.012380438950606993
train loss:0.015388305211218607
train loss:0.013297443589748666
train loss:0.04522687381419683
train loss:0.019734276260750902
train loss:0.009304941988339156
train loss:0.024036249333815055
train loss:0.017905973473614823
train loss:0.03422305927368249
train loss:0.046961798134745755
train loss:0.02080914045186422
train loss:0.007861916209133988
train loss:0.020164452744416774
=== epoch:5, train acc:0.985, test acc:0.981 ===
train loss:0.011985373528235164
tr

train loss:0.016035519153374284
train loss:0.26812418394671933
train loss:0.07344400463274896
train loss:0.008884879929105072
train loss:0.012685098737906419
train loss:0.010947003285247498
train loss:0.011846836392737562
train loss:0.010077130303426399
train loss:0.011261396595128594
train loss:0.03668300978402085
train loss:0.033050976023215065
train loss:0.0203964549409512
train loss:0.0028518209727547896
train loss:0.06010165074321191
train loss:0.00815088148946516
train loss:0.06231997306080294
train loss:0.04975724897757406
train loss:0.03310728716959116
train loss:0.004697564859667411
train loss:0.11470109383227343
train loss:0.004802856124237612
train loss:0.006903806528251224
train loss:0.01883392147157598
train loss:0.01938472773503267
train loss:0.0621665833630219
train loss:0.019642645575889753
train loss:0.0546387056817
train loss:0.01882822773856373
train loss:0.03878518301997468
train loss:0.011140126750052038
train loss:0.006829876644448386
train loss:0.0213202828528835

train loss:0.0070747419692561205
train loss:0.07940058926475571
train loss:0.04665726539931626
train loss:0.06309684620888918
train loss:0.013484856724437494
train loss:0.025646157336789353
train loss:0.10557149951893292
train loss:0.034351389238254974
train loss:0.03110746321398055
train loss:0.01069715315266806
train loss:0.07192504625896498
train loss:0.004801667638470311
train loss:0.005711691817032396
train loss:0.013286645829696988
train loss:0.058421682552726964
train loss:0.06082633429885617
train loss:0.030171735679387184
train loss:0.03054550051066266
train loss:0.02682498284511251
train loss:0.03965265784613654
train loss:0.022504383638945233
train loss:0.011870718659785067
train loss:0.03756150235605026
train loss:0.0348440432848637
train loss:0.011534111993036573
train loss:0.030615393055822445
train loss:0.003724272394656562
train loss:0.04357424893615069
train loss:0.08044441215833667
train loss:0.009897873636709582
train loss:0.02304286095328018
train loss:0.00875095158